In [1]:
import pandas as pd
import numpy as np

from pathlib import Path
from ipyleaflet import Map, CircleMarker, GeoData, FullScreenControl, basemaps, Popup
from ipywidgets import RadioButtons, HTML, interact, Button
#from utilities.map import stations, trips, geometries
path = Path('resources/data')

In [1]:
!python --version

Python 3.7.10


In [2]:
stations = pd.read_pickle(path / 'stations')
trips = pd.read_pickle(path / 'trips')
trips = trips.fillna(0)

geometries = pd.read_pickle('resources/geometry/geometries1')


In [11]:
HAS_CLICKED = False 
CURRENT_SUBSET = 'all'
CURRENT_STATION_ID = None

In [12]:
def add_paths(station_id):
    '''
    helper som legger til paths fra stasjon når clicker på marker
    '''
    trips_sub = trips.loc[CURRENT_SUBSET]
    bins = pd.cut(trips_sub.loc[station_id, 'num_trips'], bins=10, labels=range(10)) 
    for idx in geometries.loc[station_id].index:
        if trips_sub.loc[(station_id, idx)]['num_trips'] < 1:
            continue
        geom = geometries.loc[(station_id, idx)]
        if not isinstance(geom, float): # hacky måte å sjekke om nan...
            weight = 1 + 0.2*bins[idx]
            path_layer = GeoData(geo_dataframe = geom, style={'color':'black', 'weight':weight})
            m.add_layer(path_layer)

def remove_paths():
    '''
    helper som fjerne eksisterende paths når clicker på ny marker
    '''
    for x in m.layers:
        if isinstance(x, GeoData):
            m.remove_layer(x)


In [13]:
def get_duration(origin, destination):
    '''
    returnerer formatert string med gj.snitt reisetid
    '''
    trips_sub = trips.loc[CURRENT_SUBSET]
    obs = trips_sub.loc[(origin, destination)].astype(int)
    if obs['num_trips'] == 0:
        string_out = ''
    elif obs.hours > 0:
        string_out = 'Over 60 min'
    else:
        minutes = '0' + str(obs.minutes) if len(str(obs.minutes)) == 1 else str(obs.minutes)
        seconds = '0' + str(obs.seconds) if len(str(obs.seconds)) == 1 else str(obs.seconds)
        string_out = minutes + ':' + seconds
    return string_out

def make_markers(stations):
    '''
    returnere series med stasjons_id : CircleMarker objekt
    '''
    # node_ids = get_node_ids(stations) # series
    markers = pd.Series(index=stations.index, dtype='object')
    for idx in stations.index:
    #   node_id = node_ids[idx]
        location = stations.loc[idx, ['latitude','longitude']].to_list()
        marker = CircleMarker(location=location, radius=4, weight=3,
                            title = stations.loc[idx,'name'],
                            color='green', fill_opacity=0.2)
        marker.on_click(handle_click)

        marker.on_mouseover(handle_mouseover)
        marker.on_mouseout(handle_mouseout)
        markers.loc[idx] = marker
    return markers        


In [14]:
radio_buttons = RadioButtons(options=['Alle dager', 'Hverdager', 'Helg'], description='Hvilke dager')
@interact(x=radio_buttons)
def plot_df(x):
    global CURRENT_SUBSET
    table = {'Alle dager':'all', 'Hverdager':'weekdays', 'Helg':'weekends'}
    CURRENT_SUBSET = table[x]
    if HAS_CLICKED:
        remove_paths()
        add_paths(CURRENT_STATION_ID)
        

interactive(children=(RadioButtons(description='Hvilke dager', options=('Alle dager', 'Hverdager', 'Helg'), va…

In [15]:
# dropper denne tror jeg
reset_button = Button(description='Nullstill')
def reset_callback(*args, **kwargs):
    global CURRENT_SUBSET, HAS_CLICKED, CURRENT_STATION_ID
    CURRENT_SUBSET = 'all'
    HAS_CLICKED = False
    CURRENT_STATION_ID = None
    remove_paths()
reset_button.on_click(reset_callback)
#reset_button    

In [16]:
def handle_click(*args, **kwargs):
    global HAS_CLICKED, CURRENT_STATION_ID
    HAS_CLICKED = True
    y, x = kwargs['coordinates']
    # hacky løsning for å finne stasjonsid som er nærmest koordinat...
    num_idx = (np.abs(stations.latitude-y)+np.abs(stations.longitude-x)).argmin()
    station_id = stations.index[num_idx]

    CURRENT_STATION_ID = int(station_id)

    remove_paths()
    add_paths(int(station_id))

def handle_mouseover(*args, **kwargs):
    y, x = kwargs['coordinates']
    # hacky løsning for å finne stasjonsid som er nærmest koordinat...
    num_idx = (abs(stations.latitude-y)).argmin()
    station_id = stations.index[num_idx]
    
    trips_sub = trips.loc[CURRENT_SUBSET]
    if not HAS_CLICKED:
        msg = HTML(value=stations.loc[station_id, 'name'])
        popup = Popup(location=[y,x], child=msg)
        m.add_layer(popup)
    if HAS_CLICKED:
        line1 = stations.loc[station_id, 'name']
        line2 = f'Antall reiser: {trips_sub.loc[(CURRENT_STATION_ID, station_id),"num_trips"]}'
        line3 = f'Gj.snitt tid: {get_duration(CURRENT_STATION_ID, station_id)}'
        value = '<br>'.join(line for line in [line1, line2, line3])

        msg = HTML(value=value)
        popup = Popup(location=[y,x], child=msg)
        m.add_layer(popup)

def handle_mouseout(**kwargs):
    # Må finnes smartere måter ...
    for x in m.layers:
        if isinstance(x,Popup):
            m.remove_layer(x)   

In [17]:
def make_markers(stations):
    '''
    returnere series med stasjons_id : CircleMarker objekt
    '''
    # node_ids = get_node_ids(stations) # series
    markers = pd.Series(index=stations.index, dtype='object')
    for idx in stations.index:
    #   node_id = node_ids[idx]
        location = stations.loc[idx, ['latitude','longitude']].to_list()
        marker = CircleMarker(location=location, radius=4, weight=3,
                            title = stations.loc[idx,'name'],
                            color='green', fill_opacity=0.2)
        marker.on_click(handle_click)

        marker.on_mouseover(handle_mouseover)
        marker.on_mouseout(handle_mouseout)
        markers.loc[idx] = marker
    return markers        

            
def initialize_map():
    center = [60.388197,5.328564]
    zoom = 13
    m = Map(center=center, zoom=zoom)
    m.add_control(FullScreenControl())
    markers = make_markers(stations)

    for marker in markers:
        m.add_layer(marker)

    return m
m = initialize_map()


In [18]:
m

Map(center=[60.388197, 5.328564], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', …